About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News


In [10]:
#importing the dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing

In [39]:
news_dataset = pd.read_csv('/content/train.csv', engine='c', error_bad_lines=False, skiprows=[10289,14645,16988,17448,17690,17913
                                                                                              ,18131,18599,19708,19900,20129,20563])

<ipython-input-39-fb201f1aa51f>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  news_dataset = pd.read_csv('/content/train.csv', engine='c', error_bad_lines=False, skiprows=[10289,14645,16988,17448,17690,17913


In [40]:
news_dataset.shape

(20551, 5)

In [42]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [45]:
#counting the missing values in the dataset
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [44]:
#replacing the missing values with null string
news_dataset=news_dataset.fillna('')

In [46]:
#merging the author name and news title
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20546    Henry Wolff UN: Migrant Deaths in Mediterranea...
20547              Жанна Ивченко Кто любит Стивена Сигала?
20548    Benjamin Weiser and William K. Rashbaum With P...
20549    Activist Post Drone Restrictions Can Help Peac...
20550    Virgil Virgil: Making America Great Again—Or N...
Name: content, Length: 20551, dtype: object


In [47]:
#separating the data & label
x=news_dataset.drop(columns='label',axis=1)
y=news_dataset['label']

In [48]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20546  20557  UN: Migrant Deaths in Mediterranean Hit Record...   
20547  20558                          Кто любит Стивена Сигала?   
20548  20559  With Preet Bharara’s Dismissal, Storied Office...   
20549  20560  Drone Restrictions Can Help Peaceful Protester...   
20550  20561  Virgil: Making America Great Again—Or Not: The...   

                                        author  \
0                                Darrell Lucus   
1                              Daniel J. Flynn   
2                           Consortiumnews.co

In [49]:
port_stem=PorterStemmer()

In [53]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [54]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [55]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20546    henri wolff un migrant death mediterranean hit...
20547                                                     
20548    benjamin weiser william k rashbaum preet bhara...
20549    activist post drone restrict help peac protest...
20550    virgil virgil make america great establish tar...
Name: content, Length: 20551, dtype: object


In [56]:
#separating the data & label
x=news_dataset['content'].values
y=news_dataset['label'].values

In [57]:
print(x)
print(y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'benjamin weiser william k rashbaum preet bharara dismiss stori offic lose top fighter new york time'
 'activist post drone restrict help peac protest combat injustic'
 'virgil virgil make america great establish target trump top strategist']
[1 0 1 ... 0 1 0]


In [58]:
#converting the textual data into numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [59]:
print(x)

  (0, 15602)	0.2840113724652463
  (0, 13400)	0.25609656195569735
  (0, 8855)	0.3626666054748448
  (0, 8581)	0.2920040733906793
  (0, 7652)	0.2483474293659538
  (0, 6971)	0.2188869009412424
  (0, 4949)	0.23308301654125652
  (0, 3775)	0.26971044162527996
  (0, 3585)	0.3589683993550342
  (0, 2946)	0.24680171145079136
  (0, 2472)	0.37119562339985973
  (0, 266)	0.26971044162527996
  (1, 16706)	0.300843595817864
  (1, 6783)	0.19069659951194495
  (1, 5476)	0.7139966574708484
  (1, 3553)	0.26450081024363203
  (1, 2800)	0.19122345607229116
  (1, 2212)	0.3825966705643743
  (1, 1883)	0.15538062798833419
  (1, 1488)	0.2937452472086173
  (2, 15527)	0.4156515946118644
  (2, 9563)	0.4925407107794549
  (2, 5939)	0.3471615646804611
  (2, 5363)	0.3864619817692448
  (2, 3090)	0.46229267892961723
  :	:
  (20548, 10582)	0.21902557639153128
  (20548, 10249)	0.08109388432240842
  (20548, 8800)	0.23668981036550582
  (20548, 5324)	0.2903358269046699
  (20548, 4110)	0.30940206901713047
  (20548, 1470)	0.3561620

In [60]:
print(y)

[1 0 1 ... 0 1 0]


Splitting the dataset to training and test data

In [61]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [63]:
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(16440, 17033) (4111, 17033)
(16440,) (4111,)


# Training the model:  Logistic Regression

In [64]:
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

# Evaluation
# Accuracy score

In [65]:
#Accuracy score on the training data
x_train_pred=model.predict(x_train)
training_data_accuracy=accuracy_score(x_train_pred,y_train)
print("Accuracy score of training data = ",training_data_accuracy)

Accuracy score of training data =  0.9866788321167883


In [66]:
#Accuracy score of the test data
x_test_pred=model.predict(x_test)
test_data_accuracy=accuracy_score(x_test_pred,y_test)
print("Accuracy score of the test data = ",test_data_accuracy)

Accuracy score of the test data =  0.9773777669666748


# Making a predictive System




In [71]:
x_news=x_test[2001]
pred=model.predict(x_news)
print(pred)
if pred[0]==0:
  print("The news is real")
else:
  print("The news is Fake")
print(y_test[2001])

[1]
The news is Fake
1


# SUPPORT VECTOR MACHINE CLASSIFIER


In [72]:
from sklearn import svm

In [74]:
classifier=svm.SVC(kernel='linear')
classifier.fit(x_train,y_train)

SVC(kernel='linear')

In [75]:
#Accuracy score of the training data
x_train_pred=classifier.predict(x_train)
training_data_accuracy=accuracy_score(x_train_pred,y_train)
print("Accuracy score of the training data = ",training_data_accuracy)

Accuracy score of the training data =  0.9973844282238443


In [77]:
# Accuracy score of the test data
x_test_pred=classifier.predict(x_test)
test_data_accuracy=accuracy_score(x_test_pred,y_test)
print("Accuracy score of the test data = ",test_data_accuracy)

Accuracy score of the test data =  0.9912430065677451


# Building a predictive system

In [79]:
x_news=x_test[3435]
pred=classifier.predict(x_news)
print(pred)
if pred[0]==0:
  print("news is real")
else:
  print("news is fake")
print(y_test[3435])

[1]
news is fake
1
